In [1]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

import numpy as np

import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix, classification_report
import itertools

%matplotlib inline

In [8]:
batch_size = 32  # The default batch size of keras.
num_classes = 10  # Number of class for the dataset
epochs = 10

In [9]:
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 1)
50000 train samples
10000 test samples


In [10]:
# Normalize the data. Before we need to connvert data type to float for computation.
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices. This is called one hot encoding.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [11]:
#define the convnet
model = Sequential()
# CONV => RELU => CONV => RELU => POOL => DROPOUT
model.add(Conv2D(32, (3, 3), padding='same',input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# CONV => RELU => CONV => RELU => POOL => DROPOUT
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# FLATTEN => DENSE => RELU => DROPOUT
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
# a softmax classifier
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_6 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_7 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 15, 15, 64)       

In [12]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [13]:
 history = model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)

Epoch 1/10
1563/1563 [==============================] - 226s 144ms/step - loss: 1.5212 - accuracy: 0.4427 - val_loss: 1.1953 - val_accuracy: 0.5763
Epoch 2/10
1563/1563 [==============================] - 223s 143ms/step - loss: 1.1503 - accuracy: 0.5891 - val_loss: 0.9790 - val_accuracy: 0.6529
Epoch 3/10
1563/1563 [==============================] - 223s 142ms/step - loss: 1.0044 - accuracy: 0.6465 - val_loss: 0.8562 - val_accuracy: 0.7048
Epoch 4/10
1563/1563 [==============================] - 226s 145ms/step - loss: 0.9109 - accuracy: 0.6793 - val_loss: 0.8014 - val_accuracy: 0.7202
Epoch 5/10
1563/1563 [==============================] - 221s 141ms/step - loss: 0.8432 - accuracy: 0.7050 - val_loss: 0.7935 - val_accuracy: 0.7243
Epoch 6/10
1563/1563 [==============================] - 221s 141ms/step - loss: 0.7851 - accuracy: 0.7261 - val_loss: 0.7516 - val_accuracy: 0.7415
Epoch 7/10
1563/1563 [==============================] - 226s 145ms/step - loss: 0.7481 - accuracy: 0.7370 - val_

In [14]:
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

313/313 [==============================] - 10s 31ms/step - loss: 0.6733 - accuracy: 0.7685
Test loss: 0.673309862613678
Test accuracy: 0.7684999704360962


In [16]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

In [17]:
history = model.fit_generator(datagen.flow(x_train, y_train,
                                    batch_size=batch_size),
                                    epochs=epochs,
                                    validation_data=(x_test, y_test),
                                    workers=4)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
1563/1563 [==============================] - 243s 155ms/step - loss: 0.9096 - accuracy: 0.6830 - val_loss: 0.7318 - val_accuracy: 0.7514
Epoch 2/10
1563/1563 [==============================] - 245s 157ms/step - loss: 0.8717 - accuracy: 0.6987 - val_loss: 0.6962 - val_accuracy: 0.7588
Epoch 3/10
1563/1563 [==============================] - 250s 160ms/step - loss: 0.8485 - accuracy: 0.7073 - val_loss: 0.6987 - val_accuracy: 0.7602
Epoch 4/10
1563/1563 [==============================] - 246s 157ms/step - loss: 0.8379 - accuracy: 0.7087 - val_loss: 0.7113 - val_accuracy: 0.7565
Epoch 5/10
1563/1563 [==============================] - 248s 159ms/step - loss: 0.8243 - accuracy: 0.7139 - val_loss: 0.6676 - val_accuracy: 0.7663
Epoch 6/10
1563/1563 [==============================] - 246s 157ms/step - loss: 0.8184 - accuracy: 0.7172 - val_loss: 0.7212 - val_accuracy: 0.7500
Epoch 7/10
1563/1563 [==============